In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from hidmed import *
import pickle
import matplotlib.pyplot as plt

## Tune hyperparameters for each setup and sample size combination

In [3]:
def tune_parameters(xdim, zdim, wdim, mdim, udim, setup, n, folds=2, seed=0):
    """Tune the parameters of the estimators for a given setup."""
    assert setup in ["a", "b", "c"], "Invalid setup. Must be 'a', 'b', or 'c'."
    datagen = LinearHidMedDGP(xdim, zdim, wdim, mdim, udim, setup=setup, seed=seed)

    # tune all hyperparameters using setup "c"
    estimator = ProximalMultiplyRobust(
        generalized_model=True,
        folds=folds,
        num_runs=200,
        n_jobs=1,
    )
    dataset = datagen.sample_dataset(n, seed=seed + 1)
    estimator.fit(dataset)
    return estimator.params


# tuned_parameters = {}
# for n in np.arange(300, 5000, 600):
#     # 1d-case
#     print("1d-case, n =", n)
#     tuned_parameters[1, n] = tune_parameters(1, 1, 1, 1, 1, "c", n, folds=2)
#     # 5d-case
#     print("5d-case, n =", n)
#     tuned_parameters[5, n] = tune_parameters(5, 2, 2, 2, 1, "c", n, folds=2)
#     print("\n")

# pickle.dump(tuned_parameters, open("assets/tuned_parameters.pkl", "wb"))

In [38]:
params = tune_parameters(1, 1, 1, 1, 1, "c", 1500, folds=2)
params

==== Cross-fitting fold 1 (500/1000 fit/eval)
Bridge q params: {'lambda1': 0.0012710977557546606, 'lambda2': 0.0005000000000000001, 'gamma1': 0.0010000000000000002, 'gamma2': 0.0037275937203149418}, score: 0.4704905931019196
Bridge h params: {'lambda1': 0.008214797213603475, 'lambda2': 92.61606457597674, 'gamma1': 0.0012067926406393284, 'gamma2': 0.0012067926406393284}, score: 0.0049210404873084
eta params: {'alpha': 3.083524658424441e-06, 'gamma': 0.0010000000000000002}, r2: 0.9716764711598869


/Users/alan/.pyenv/versions/3.11.5/envs/dev3/lib/python3.11/site-packages/sklearn/base.py:1474: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/Users/alan/.pyenv/versions/3.11.5/envs/dev3/lib/python3.11/site-packages/sklearn/base.py:1474: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/Users/alan/.pyenv/versions/3.11.5/envs/dev3/lib/python3.11/site-packages/sklearn/base.py:1474: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/Users/alan/.pyenv/versions/3.11.5/envs/dev3/lib/python3.11/site-packages/sklearn/base.py:1474: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibl

Treatment prob params: {'C': 10000000.0, 'degree': 1} log_loss:  0.69
==== Estimate 1: 0.39757677232964467, 1000 values
==== Cross-fitting fold 2 (500/1000 fit/eval)
Bridge q params: {'lambda1': 0.0012710977557546606, 'lambda2': 0.0005000000000000001, 'gamma1': 0.0010000000000000002, 'gamma2': 0.0037275937203149418}, score: nan
Bridge h params: {'lambda1': 0.008214797213603475, 'lambda2': 92.61606457597674, 'gamma1': 0.0012067926406393284, 'gamma2': 0.0012067926406393284}, score: nan
eta params: {'alpha': 3.083524658424441e-06, 'gamma': 0.0010000000000000002}, r2: nan
Treatment prob params: {'C': 10000000.0, 'degree': 1} log_loss:  nan
==== Estimate 2: 0.2978168730842236, 1000 values
==== ProximalMultiplyRobust estimate: 0.34769682270693414, 2000 values


{'h': {'lambda1': 0.008214797213603475,
  'lambda2': 92.61606457597674,
  'gamma1': 0.0012067926406393284,
  'gamma2': 0.0012067926406393284},
 'q': {'lambda1': 0.0012710977557546606,
  'lambda2': 0.0005000000000000001,
  'gamma1': 0.0010000000000000002,
  'gamma2': 0.0037275937203149418},
 'eta': {'alpha': 3.083524658424441e-06, 'gamma': 0.0010000000000000002},
 'treatment': {'C': 10000000.000000006, 'degree': 1}}

## Run estimators with chosen hyperparamters

In [39]:
datagen = LinearHidMedDGP(1, 1, 1, 1, 1, setup="c", seed=0)
dataset = datagen.sample_dataset(1500, seed=1)
datagen.true_psi()

0.32584216852328096

In [40]:
pmr = ProximalMultiplyRobust(generalized_model=True, folds=2, verbose=False, **params)
pmr.fit(dataset)

0.3096681412269156

In [41]:
pipw = ProximalInverseProbWeighting(
    generalized_model=True, folds=2, verbose=False, **params
)
pipw.fit(dataset)

0.19143405035180222

In [42]:
por = ProximalOutcomeRegression(
    generalized_model=True, folds=2, verbose=False, **params
)
por.fit(dataset)

0.3113029206423572